In [ ]:
# For data generation
import sys
# !{sys.executable} -m pip install xgboost
import shutil
import os
n_jobs = 16
os.environ["OMP_NUM_THREADS"] = str(n_jobs)
import joblib
import json

import numpy as np
from glob import glob
import re
import pandas as pd
import pickle
#random
from time import perf_counter

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,root_mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

#Plotting
import seaborn as sns
sns.set_style()
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 200

import warnings
# warnings.filterwarnings("ignore", category=np.DeprecationWarning) 


from tqdm.auto import tqdm
# from tqdm.notebook import tqdm_notebook as tqdm

In [ ]:
# with open('linear_train.bin','rb') as f:
#     train = joblib.load(f)

# with open('linear_test.bin','rb') as f:
#     test = joblib.load(f)

# with open('linear_scaler.bin','rb') as f:
#     scaler = joblib.load(f)
# X_train, y_train = train['X'],train['y']
# X_test, y_test = test['X'],test['y']


# with open('PCA5_0.8_Morgan_train.bin','rb') as f:
#     bse_train = joblib.load(f)

# with open('PCA5_0.8_Morgan_test.bin','rb') as f:
#     bse_test = joblib.load(f)

# with open('PCA5_0.8_Morgan_scaler.bin','rb') as f:
#     bse_scaler = joblib.load(f)

# X_bse_train, y_bse_train = bse_train['X'],bse_train['y']
# X_bse_test, y_bse_test = bse_test['X'],bse_test['y']


# X_bse_train[np.isclose(X_bse_train,1)]=1
# X_bse_train[np.isclose(X_bse_train,-1)]=-1



# X_train, y_train = X_bse_train, y_bse_train
# X_test, y_test = X_bse_test, y_bse_test
# scaler = bse_scaler



with open('0.1_5_DDCC_train.bin','rb') as f:
    ddcc_train = joblib.load(f)

with open('0.1_5_DDCC_test.bin','rb') as f:
    ddcc_test = joblib.load(f)

with open('0.1_5_DDCC_scaler.bin','rb') as f:
    ddcc_scaler = joblib.load(f)

X_ddcc_train, y_ddcc_train = ddcc_train['X'],ddcc_train['y']
X_ddcc_test, y_ddcc_test = ddcc_test['X'],ddcc_test['y']

X_ddcc_train = X_ddcc_train.reshape(19,64,5)[0:1].reshape(-1,5)
X_ddcc_test = X_ddcc_test.reshape(-1,64,5)[0:1].reshape(-1,5)
y_ddcc_train = y_ddcc_train.reshape(19,64)[0:1].flatten()
y_ddcc_test = y_ddcc_test.reshape(-1,64)[0:1].flatten()


# X_train, y_train = X_ddcc_train, y_ddcc_train
# X_test, y_test = X_ddcc_test, y_ddcc_test
X_train, y_train = X_ddcc_train, y_ddcc_train
X_test, y_test = X_ddcc_test, y_ddcc_test
scaler = ddcc_scaler

print(X_train.shape, X_test.shape)

In [ ]:
X_train.shape

In [ ]:
from xgboost import XGBRegressor
kwargs = {'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 10,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': np.nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': 0.001,
 'reg_lambda': 0.1,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

# kwargs = dict(pd.read_excel("iron_VDZP_params.xlsx").values)
model=XGBRegressor(**kwargs)
model.fit(X_train,y_train)
y_pred_train=model.predict(X_train)
y_pred_test=model.predict(X_test)
print(f"R2: {r2_score(y_train,y_pred_train):.4f},{r2_score(y_test,y_pred_test):.4f}")
print(f"RMSE (mEh): {root_mean_squared_error(y_train,y_pred_train)*1e3:.4f},{root_mean_squared_error(y_test,y_pred_test)*1e3:.4f}")

# params = {'max_depth': [1, 10, 100],
#           'n_estimators': [100, 500, 1000],
#           'reg_lambda': [1e-6, 1e-3,1e-1],
#           'reg_alpha': [1e-6, 1e-3,1e-1]}

# model = XGBRegressor()
# grid = GridSearchCV(estimator=model, 
#                    param_grid=params,
#                    scoring='r2', 
#                    verbose=1000,n_jobs=12).fit(X_train,y_train)


# model=grid.best_estimator_
# y_pred_train=model.predict(X_train)
# y_pred_test=model.predict(X_test)
# print(f"R2: {r2_score(y_train,y_pred_train):.4f},{r2_score(y_test,y_pred_test):.4f}")
# print(f"RMSE (mEh): {root_mean_squared_error(y_train,y_pred_train)*1e3:.4f},{root_mean_squared_error(y_test,y_pred_test)*1e3:.4f}")

In [ ]:
plt.scatter(y_train,y_pred_train)
plt.scatter(y_test,y_pred_test)
plt.show()